In [4]:
import cv2
# import time
import base64
import numpy as np
# from src.utils.utils import load_graph, load_labels
# from src.stamp_detection.stamp_roi_classifier.stamp_roi_classifier import DetectStamp

def generate_grids(img, nRows, mCols):

    # Dimensions of the image
    sizeX = img.shape[1]
    sizeY = img.shape[0]
    # iter = 0
    # print(img.shape)
    roi_list = []
    for i in range(0, nRows):
        for j in range(0, mCols):
            roi = img[int(i * sizeY / nRows):i * int(sizeY / nRows) + int(sizeY / nRows),
                  j * int(sizeX / mCols):j * int(sizeX / mCols) + int(sizeX / mCols)]
            # cv2.imshow('rois'+str(i)+str(j), roi)
            roi_list.append(roi)

    return [200, roi_list]
def generate_sequence(self, img_base64, roi_classifier, roi_classifier_graph, rows, cols):

        img_bytes = base64.b64decode(img_base64)
        img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), -1)
        # start_time = time.time()
        status, roi_list = self.generate_grids(img, rows, cols)
        # end_time = time.time()
        # print(str(end_time-start_time))
        # print("ROI LIst", roi_list)
        # print(len(roi_list))
        datalist = []
        # for roi in roi_list:
        #     flag, res, conf = roi_classifier.predictROI(roi, roi_classifier_graph)
        #     datalist.append(res[0])

        multi_input = []
        for idx, roi in enumerate(roi_list):
            multi_input.append((idx, roi, roi_classifier_graph))
            # flag, res, conf = roi_classifier.predictROI((roi, roi_classifier_graph))
            # datalist.append(res[0])

        # p = Pool(processes=4)
        p = ThreadPool()
        results = p.map(roi_classifier.predictROI, multi_input)
        # print(results)

        for result in results:
            datalist.append(result[1][0])

        sequence_list = [str(i) for i in datalist]
        sequence = (",".join(sequence_list)).strip()

        return [200, sequence]



In [ ]:
def shipper_detect_signature(img_base64, roi_classifier, roi_classifier_graph, rows, cols):
    """

    :param img_base64:
    :param roi_classifier:
    :param roi_classifier_graph:
    :param rows:
    :param cols:
    :return:
    """
    result = None
    img_bytes = base64.b64decode(img_base64)
    img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), -1)
    # img = img_base64
    status, roi_list = generate_grids(img, rows, cols)
    # roi_list.reverse()
    roi = roi_list[-1]
    # print(roi.shape)
    (y, x, _) = roi.shape
    new_y = int(y * (50 / 100))
    # print(new_y)
    roi = roi[:new_y, :]
    # plt.imshow(roi)
    # plt.title('roi')
    # plt.show()
    resultflag, res, conf = roi_classifier.predictROI(roi, roi_classifier_graph)
    # print(res)
    if res[0] == 1:
        conf_score = float(round(conf.max(),2))#format(conf[1], "%")
        # print("Identified")
        result = {
            "result": True,
            "conf_score": conf_score
        }
    elif res[1] == 1:
        conf_score = float(round(conf.max(), 2))  # format(conf[1], "%")
        # print("Identified")
        result = {
            "result": False,
            "conf_score": conf_score
        }
    return [200, result]

def nestle_detect_signature(img_base64, roi_classifier, roi_classifier_graph, rows, cols):
    """

    :param img_base64:
    :param roi_classifier:
    :param roi_classifier_graph:
    :param rows:
    :param cols:
    :return:
    """
    result = None
    img_bytes = base64.b64decode(img_base64)
    img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), -1)
    # img = img_base64
    status, roi_list = generate_grids(img, rows, cols)
    roi_list.reverse()
    roi_list = roi_list[:3]
    for idx, roi in enumerate(roi_list):
        print(idx)
        cv2.imwrite('sign-test_' + str(idx) + '_roi.jpg', roi)
        resultflag, res, conf = roi_classifier.predictROI(roi, roi_classifier_graph)
        # print(res)
        if res[0] == 1:
            cv2.imwrite('detected_roi.jpg', roi)
            conf_score = float(round(conf.max(), 2))  # format(conf[1], "%")
            # print("Identified")
            result = {
                "result": True,
                "conf_score": conf_score
            }
            return [200, result]
        # print("Not Identified")
    result = {
        "result": False
    }
    return [200, result]

def detect_any_signature(img_base64, roi_classifier, roi_classifier_graph, rows, cols):
    """

    :param img_base64:
    :param roi_classifier:
    :param roi_classifier_graph:
    :param rows:
    :param cols:
    :return:
    """
    result = None
    img_bytes = base64.b64decode(img_base64)
    img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), -1)
    # img = img_base64
    status, roi_list = generate_grids(img, rows, cols)
    roi_list.reverse()

    for idx, roi in enumerate(roi_list):
        print(idx)
        resultflag, res, conf = roi_classifier.predictROI(roi, roi_classifier_graph)
        # print(res)
        if res[0] == 1:
            cv2.imwrite('detected_roi.jpg', roi)
            conf_score = float(round(conf.max(), 2))  # format(conf[1], "%")
            # print("Identified")
            result = {
                "result": True,
                "conf_score": conf_score
            }
            return [200, result]
        # print("Not Identified")
    result = {
        "result": False
    }
    return [200, result]

def nestle_detect_signature(img_base64, roi_classifier, roi_classifier_graph, rows, cols):
    """

    :param img_base64:
    :param roi_classifier:
    :param roi_classifier_graph:
    :param rows:
    :param cols:
    :return:
    """
    result = None
    img_bytes = base64.b64decode(img_base64)
    img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), -1)
    # img = img_base64
    status, roi_list = generate_grids(img, rows, cols)
    roi_list.reverse()
    roi_list = roi_list[:3]
    for idx, roi in enumerate(roi_list):
        print(idx)
        cv2.imwrite('sign-test_' + str(idx) + '_roi.jpg', roi)
        resultflag, res, conf = roi_classifier.predictROI(roi, roi_classifier_graph)
        # print(res)
        if res[0] == 1:
            cv2.imwrite('detected_roi.jpg', roi)
            conf_score = float(round(conf.max(), 2))  # format(conf[1], "%")
            # print("Identified")
            result = {
                "result": True,
                "conf_score": conf_score
            }
            return [200, result]
        # print("Not Identified")
    result = {
        "result": False
    }
    return [200, result]


In [ ]:
def pepsi_detect_signature(img_base64, roi_classifier, roi_classifier_graph, rows, cols):
    """

    :param img_base64:
    :param roi_classifier:
    :param roi_classifier_graph:
    :param rows:
    :param cols:
    :return:
    """
    result = None
    image_bytes = base64.b64decode(img_base64)
    img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), -1)
    status, roi_list = generate_grids(img, rows, cols)
    roi_list.reverse()
    